# FNN Linus Etemi

## Imports


In [122]:
TYPE = 'CPU'

if TYPE == 'CPU':
    import numpy as np
else:
    import cupy as np
    
import scipy.special
import math
import matplotlib
from ray import tune
from csv import writer


## Config

In [117]:
training_data_file = open("./MNISTData_red/mnist_train_red_8_9_by_90P.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

test_data_file = open("./mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

hiddennodes_start = 800
hiddennodes_stop = 800
hiddennodes_step = 100

learningrate_start = 0.08
learningrate_stop = 0.08
learningrate_step = 0.01

epochs_start = 1
epochs_stop = 2
epochs_step = 0

outputnodes = 10
hiddennodes=800
learningrate=0.08
epochs=5

label_min = 0.80

## Neuronales Netz 

In [118]:
class neuralNetwork:
    def __init__(self, innodes, outnodes, hidnodes, learningrate, epochs, network_number):
        self.inodes = innodes
        self.onodes = outnodes
        self.hnodes = hidnodes
        self.lr = learningrate
        self.epo = epochs
        self.net = network_number
        self.wih = ((np.random.rand(self.hnodes,self.inodes)*2)-1)
        self.who = ((np.random.rand(self.onodes,self.hnodes)*2)-1)
        
    def activation_function(self, z):
        return 1/(1+ np.exp(-z))

        pass

    def train(self, inputs_list, targets_list):
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T

        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)

        final_inputs = np.dot(self.who, hidden_outputs)

        final_outputs = self.activation_function(final_inputs)
        
        output_errors = targets - final_outputs
        hidden_errors = np.dot(self.who.T, output_errors) 
        self.who += self.lr * np.dot( (output_errors*final_outputs*(1.0-final_outputs)), np.transpose(hidden_outputs)) 
        self.wih += self.lr * np.dot( (hidden_errors*hidden_outputs*(1.0-hidden_outputs)), np.transpose(inputs)) 
        
        pass

    def query(self, input_list):
        inputs = np.array(input_list, ndmin=2).T
        
        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs
    
    
    def test_train(self):  
        if(self.net == 1):
            training_data_file = open("./MNISTData_red/mnist_train_red_8_9_by_90P.csv", 'r')
            training_data_list = training_data_file.readlines()
            training_data_file.close()
        elif(self.net == 2):
            training_data_file = open("./mnist_second_train.csv", 'r')
            training_data_list = training_data_file.readlines()
            training_data_file.close()

        for e in range(self.epo):
            for record in training_data_list:

                all_values = record.split(',')

                inputs = (np.asarray(all_values[1:], dtype=np.float64) / 255.0 * 0.99) + 0.01

                targets = np.zeros(outputnodes) + 0.01
                targets[int(all_values[0])] = 0.99
                self.train(inputs, targets)
                pass
            pass

        scorecard = []
        save = []

        for record in test_data_list:
            all_values = record.split(',')
            correct_label = int(all_values[0])
            inputs = (np.asarray(all_values[1:], dtype=np.float64) / 255.0 * 0.99) + 0.01
            outputs = self.query(inputs)
            label = np.argmax(outputs)

            if(label >= label_min):
                
                scorecard.append(1)
            else:

                scorecard.append(0)
                if(self.net == 1):
                    save.append(list(map(int, all_values)))
                else:
                    pass
                pass
            

            scorecard_array = np.asarray(scorecard)
            performance = scorecard_array.sum() / scorecard_array.size
        
        if(self.net == 1):
            np.savetxt('mnist_second_train.csv', save, fmt='%d', delimiter=',')
        else:
            pass

        return performance

## CSV Writer und Grid Search

In [119]:
def append_list_as_row(list_of_elem):
    with open('Results.csv', 'a+', newline='') as write_obj:
        csv_writer = writer(write_obj)
        csv_writer.writerow(list_of_elem)

In [120]:
#learningrate_i = learningrate_start
#hiddennodes_i = hiddennodes_start
#epochs_i = epochs_start


#while epochs_i <= epochs_stop:
#    while hiddennodes_i <= hiddennodes_stop:
#        while learningrate_i <= learningrate_stop:
#            intermediate_score = neuralNetwork(784,10,hiddennodes_i,learningrate_i,epochs_i).test_train()
#            print('Epoch [{:02d}/{:02d}]     | Hidden_Nodes [{:04d}/{:04d}]     | Learning_Rate [{:.2f}/{:.2f}]     | Performance: {:.5f}'.format(epochs_i,epochs_stop,hiddennodes_i,hiddennodes_stop,learningrate_i,learningrate_stop,intermediate_score))
#            append_list_as_row([784,10,hiddennodes_i,learningrate_i,epochs_i,intermediate_score])
#            learningrate_i += learningrate_step 
#        learningrate_i = learningrate_start    
#        hiddennodes_i += hiddennodes_step
#        print("========================================================")
#    hiddennodes_i = hiddennodes_start
#    epochs_i += epochs_step
#    print("========================================================")

## Scoring und Aufruf von Beiden Netzen

In [121]:

network_number=1
intermediate_score = neuralNetwork(784,10,hiddennodes,learningrate,epochs,network_number).test_train()
print(intermediate_score)

network_number=2
intermediate_score = neuralNetwork(784,10,hiddennodes,learningrate,epochs,network_number).test_train()
print(intermediate_score)

0.8951
0.5243
